In [37]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
def GetLatest():
  url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
  parameters = {
    'start':'1',
    'limit':'5000',
    'convert':'USD'
  }
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return data

In [38]:
def GetMap(id):
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/category'
  parameters = {
    'id':f'{id}',
  }
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    map = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return map

In [39]:
def GetCategories():
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/categories'
  parameters = {}
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'c0fef76c-7d4e-43e1-bdfa-3b02bead65e0',
  }

  session = Session()
  session.headers.update(headers)

  try:
    response = session.get(url, params=parameters)
    categories = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  return categories

In [61]:
import pandas as pd
import random
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

def Streaming():
    DATA = []
    categories = GetCategories()
    Dict = {
        "CategoryName" :None,
        "CoinName" :None,
        "NumMarketPairs" :None,
        "DateAdded" :None,
        "IsActive" :None,
        "CMCRank" :None,
        "PriceUSD" :None,
        "Volume24" :None,
        "PercentChange1h" :None,
        "PercentChange24h" :None,
        "PercentChange7d" :None,
        "MarketCap" :None,
        "LastUpdated" :None
    }
    for i in range(len(categories['data'])):
        ID = categories['data'][i]['id']
        map = GetMap(ID)
        try:
            for j in range(len(map["data"]["coins"])):
                    Dict["CategoryName"] = categories['data'][j]['name']
                    Dict["CoinName"] = map['data']['coins'][0]['name']
                    Dict["NumMarketPairs"] = map['data']['coins'][0]['num_market_pairs']
                    Dict["DateAdded"] = map['data']['coins'][0]['date_added']
                    Dict["IsActive"] = map['data']['coins'][0]['is_active']
                    Dict["CMCRank"] = map['data']['coins'][0]['cmc_rank']
                    Dict["PriceUSD"] = map['data']['coins'][0]["quote"]["USD"]['price']
                    Dict["Volume24"] = map['data']['coins'][0]["quote"]["USD"]['volume_24h']
                    Dict["PercentChange1h"] = map['data']['coins'][0]["quote"]["USD"]['percent_change_1h']
                    Dict["PercentChange24h"] = map['data']['coins'][0]["quote"]["USD"]['percent_change_24h']
                    Dict["PercentChange7d"] =  map['data']['coins'][0]["quote"]["USD"]['percent_change_7d']
                    Dict["MarketCap"] = map['data']['coins'][0]["quote"]["USD"]['market_cap']
                    Dict["LastUpdated"] = map['data']['coins'][0]["quote"]["USD"]['last_updated']
                    DATA.append(Dict)
                    df = spark.createDataFrame(DATA)
                    df.show()
        except Exception as e:
            print(e)
            continue  

In [62]:
Streaming()

+-------+--------------------+--------+--------------------+--------+--------------------+--------------------+--------------+---------------+----------------+---------------+----------------+--------------------+
|CMCRank|        CategoryName|CoinName|           DateAdded|IsActive|         LastUpdated|           MarketCap|NumMarketPairs|PercentChange1h|PercentChange24h|PercentChange7d|        PriceUSD|            Volume24|
+-------+--------------------+--------+--------------------+--------+--------------------+--------------------+--------------+---------------+----------------+---------------+----------------+--------------------+
|     32|eGirl Capital Por...|Lido DAO|2020-12-15T00:00:...|       1|2023-11-21T15:56:...|2.1286336287513595E9|           262|     2.06839991|     -5.51912488|     8.55195739|2.39121213063121|1.1363795641767639E8|
+-------+--------------------+--------+--------------------+--------+--------------------+--------------------+--------------+---------------+--

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\LENOVO\anaconda3\envs\spark\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\anaconda3\envs\spark\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\anaconda3\envs\spark\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [1]:
# from pyspark.sql.functions import from_json, col
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# sc = SparkContext()
# spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()


In [16]:
import pandas as pd
import json
import matplotlib.pyplot as plt
# Dict = pd.read_csv("CoinMarketCap.csv").to_dict()
# while True:
#     try:    
#         data_json = json.dumps(Dict)
#         df = spark.read.json(sc.parallelize([data_json]))
#         df.show()
#         df.writeStream.outputMode("append").format("console").start().awaitTermination()
#     except Exception as e:
#         print(e)
#         continue

In [17]:
df = pd.read_csv("CoinMarketCap.csv")
df.head()

,CategoryName,CoinName,NumMarketPairs,DateAdded,IsActive,CMCRank,PriceUSD,Volume24,PercentChange1h,PercentChange24h,PercentChange7d,MarketCap,LastUpdated
0,BRC-20,ORDI,72,2023-05-08T05:51:24.000Z,1,90,23.216773,1.791483e+08,1.080994,-3.889371,11.136833,4.875522e+08,2023-11-19T09:54:00.000Z
1,Modular Blockchain,ORDI,72,2023-05-08T05:51:24.000Z,1,90,23.216773,1.791483e+08,1.080994,-3.889371,11.136833,4.875522e+08,2023-11-19T09:54:00.000Z
2,Toncoin Ecosystem,ORDI,72,2023-05-08T05:51:24.000Z,1,90,23.216773,1.791483e+08,1.080994,-3.889371,11.136833,4.875522e+08,2023-11-19T09:54:00.000Z
3,Friend Tech,ORDI,72,2023-05-08T05:51:24.000Z,1,90,23.216773,1.791483e+08,1.080994,-3.889371,11.136833,4.875522e+08,2023-11-19T09:54:00.000Z
4,SEI Ecosystem,ORDI,72,2023-05-08T05:51:24.000Z,1,90,23.216773,1.791483e+08,1.080994,-3.889371,11.136833,4.875522e+08,2023-11-19T09:54:00.000Z


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4309 entries, 0 to 4308
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CategoryName      4309 non-null   object 
 1   CoinName          4309 non-null   object 
 2   NumMarketPairs    4309 non-null   int64  
 3   DateAdded         4309 non-null   object 
 4   IsActive          4309 non-null   int64  
 5   CMCRank           4309 non-null   int64  
 6   PriceUSD          4309 non-null   float64
 7   Volume24          4309 non-null   float64
 8   PercentChange1h   4309 non-null   float64
 9   PercentChange24h  4309 non-null   float64
 10  PercentChange7d   4309 non-null   float64
 11  MarketCap         4309 non-null   float64
 12  LastUpdated       4309 non-null   object 
dtypes: float64(6), int64(3), object(4)
memory usage: 437.8+ KB


In [50]:
# df.loc[:,["CoinName","PriceUSD","Volume24","PercentChange1h","PercentChange24h","PercentChange7d","MarketCap"]].groupby("CoinName").max().sort_values(by="PriceUSD",ascending=False).plot(kind="bar",figsize=(20,10))

In [49]:
# Modifier le dictionnaire Dict avec des valeurs aléatoires
import random
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()
Dict = {
    "CategoryName" : random.choice(["Bitcoin", "Ethereum", "Ripple"]),
    "CoinName" : random.choice(["BTC", "ETH", "XRP"]),
    "NumMarketPairs" : random.randint(100, 1000),
    "DateAdded" : random.choice(["2021-01-01", "2021-02-01", "2021-03-01"]),
    "IsActive" : random.choice([True, False]),
    "CMCRank" : random.randint(1, 100),
    "PriceUSD" : random.uniform(0, 100000),
    "Volume24" : random.uniform(0, 1000000000),
    "PercentChange1h" : random.uniform(-10, 10),
    "PercentChange24h" : random.uniform(-50, 50),
    "PercentChange7d" : random.uniform(-100, 100),
    "MarketCap" : random.uniform(0, 1000000000000),
    "LastUpdated" : random.choice(["2021-04-01", "2021-04-02", "2021-04-03"])
}

# Créer une liste de dictionnaires avec des valeurs
data = [Dict for _ in range(10)]

# Créer un dataframe à partir de la liste de dictionnaires
df = spark.createDataFrame(data)

# Afficher le dataframe
df.show()


+-------+------------+--------+----------+--------+-----------+--------------------+--------------+-----------------+-----------------+------------------+-----------------+-------------------+
|CMCRank|CategoryName|CoinName| DateAdded|IsActive|LastUpdated|           MarketCap|NumMarketPairs|  PercentChange1h| PercentChange24h|   PercentChange7d|         PriceUSD|           Volume24|
+-------+------------+--------+----------+--------+-----------+--------------------+--------------+-----------------+-----------------+------------------+-----------------+-------------------+
|     14|    Ethereum|     BTC|2021-01-01|   false| 2021-04-03|2.821360102475694E11|           101|6.205623524382652|25.50776189233804|12.070595159617241|97526.51137090888|7.861311622127223E8|
|     14|    Ethereum|     BTC|2021-01-01|   false| 2021-04-03|2.821360102475694E11|           101|6.205623524382652|25.50776189233804|12.070595159617241|97526.51137090888|7.861311622127223E8|
|     14|    Ethereum|     BTC|2021